# Load Data

In [1]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
import DC_Pickle as dcp
import Curve_Functions as cv
import matplotlib.pyplot as plt
%matplotlib inline

def disp_Data(x, y_true, y_pred, file_path, clt_num, cost, rows=1, columns=1, size=(6, 4)):
    fig, ax = plt.subplots(rows, columns, figsize=size)
    ax.plot(x, y_true, 'rx', label='average score')
    ax.plot(x, y_pred, 'b-', label='curve fitting')
    ax.set_xlim([0, max(x)+1])
    ax.set_ylim([0, max(y_true)+0.2])
    ax.legend(fontsize=14)
    ax.set_title("cluster {0}: cost {1}".format(clt_num, round(cost, 2)))
    fig.savefig(file_path, dpi=100)
    plt.show()
    
train_idx = dcp.open_Pickle("../../data/pickles/clusters_origin/15attempts/indices/index12.pickle")
train_idx = train_idx[9] #2, 8

train_scores = dcp.open_Pickle('../../data/pickles/seperate_origin/eventValue.pickle')
train_scores = train_scores[:300, :]/1e+4
attempts15 = np.arange(15)+1
attempts300 = np.arange(300)+1

## filtering discontinuous under 15 attempts
idx_all = []
idx_pure = []

for i in range(train_scores.shape[1]):
    if not np.isnan(train_scores[:15, i]).any():
        idx_all.append(i)
        idx_pure.append(i)
    else:
        idx_all.append(np.nan)

train_scores = train_scores[:, idx_pure]
print("Training data set: {0}".format(np.shape(train_scores)))

## get cluster data and cluster average data (average is centroid)
nClt = 12
for i in range(nClt):
    name = "cluster{0}".format(i+1)
    globals()[name] = train_scores[:, train_idx==i] # get cluster data
    
    ## get cluster average data
    data = np.ones(300)
    for j in range(len(data)):
        avg = eval(name)[j, :]
        avg = np.sum(avg[~np.isnan(avg)])/len(avg[~np.isnan(avg)])
        data[j] = avg

    globals()["avg{0}".format(i+1)] = data
    
dcp.make_folders("../../Figs/curve_fitting/")

Training data set: (300, 22832)
../../Figs/curve_fitting/ already present - Skipping pickling.


# 1. Single Curve
## 1-1. Two parameters

In [ ]:
dcp.make_folders("../../Figs/curve_fitting/exponential2/single/")

seed = [1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    ## train
    exp2_opt, exp2_cost = cv.curve_Fitting(
        cv.exponential_least2, cv.exponential_curve2, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed, 
        "../../Figs/curve_fitting/exponential2/single/sample{0}".format(i+1), clt_num = i+1)
    
    true_opt, true_cost = cv.curve_Fitting(
        cv.exponential_least2, cv.exponential_curve2, 
        attempts300, eval("avg{0}".format(i+1)), seed, 
        "../../Figs/curve_fitting/exponential2/single/dummy", clt_num = i+1)
    
    ## validation
    y_fit_valid = cv.exponential_curve2(attempts300, exp2_opt[0], exp2_opt[1])
    y_fit_true = cv.exponential_curve2(attempts300, true_opt[0], true_opt[1])
    
    exp2_cost_valid = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_valid) # get cost for all data
    exp2_cost_true = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_true) # get cost for all data
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_valid, 
              file_path="../../Figs/curve_fitting/exponential2/single/valid{0}".format(i+1), 
              clt_num=i+1, cost = exp2_cost_valid)
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_true, 
          file_path="../../Figs/curve_fitting/exponential2/single/true{0}".format(i+1), 
          clt_num=i+1, cost = exp2_cost_true)

## 1-2. Three parameters

In [ ]:
dcp.make_folders("../../Figs/curve_fitting/exponential3/single/")

seed = [1, 1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    exp3_opt, exp3_cost = cv.curve_Fitting(
        cv.exponential_least3, cv.exponential_curve3, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "../../Figs/curve_fitting/exponential3/single/{0}".format(i+1), clt_num = i+1)
    
    true_opt, true_cost = cv.curve_Fitting(
        cv.exponential_least3, cv.exponential_curve3, 
        attempts300, eval("avg{0}".format(i+1)), seed,
        "../../Figs/curve_fitting/exponential3/single/dummy".format(i+1), clt_num = i+1)

    y_fit_valid = cv.exponential_curve3(attempts300, exp3_opt[0], exp3_opt[1], exp3_opt[2])
    y_fit_true = cv.exponential_curve3(attempts300, true_opt[0], true_opt[1], true_opt[2])
    
    exp3_cost_valid = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_valid) # get cost for all data
    exp3_cost_true = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_true) # get cost for all data
    
    cost_valid = cv.cost_Function(attempts300, y_fit_valid)
    cost_true = cv.cost_Function(attempts300, y_fit_true)
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_valid,
              file_path="../../Figs/curve_fitting/exponential3/single/valid{0}".format(i+1), 
              clt_num=i+1, cost = exp3_cost_valid)
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_true,
              file_path="../../Figs/curve_fitting/exponential3/single/true{0}".format(i+1), 
              clt_num=i+1, cost = exp3_cost_true)

## 1-2. Four parameters

In [ ]:
dcp.make_folders("../../Figs/curve_fitting/exponential4/single/")

seed = [1, 1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    exp3_opt, exp3_cost = cv.curve_Fitting(
        cv.exponential_least3, cv.exponential_curve3,
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "../../Figs/curve_fitting/exponential3/single/sample{0}".format(i+1), clt_num = i+1)

    true_opt, true_cost = cv.curve_Fitting(
        cv.exponential_least3, cv.exponential_curve3, 
        attempts300, eval("avg{0}".format(i+1)), seed,
        "../../Figs/curve_fitting/exponential3/single/dummy", clt_num = i+1)

    y_fit_valid = cv.exponential_curve3(attempts300, exp3_opt[0], exp3_opt[1], exp3_opt[2])
    y_fit_valid = cv.exponential_curve3(attempts300, true_opt[0], true_opt[1], true_opt[2])
    
    exp3_cost_valid = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_valid) # get cost for all data
    exp3_cost_true = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit_true) # get cost for all data
    
    cost_valid = cv.cost_Function(attempts300, y_fit_valid)
    cost_true = cv.cost_Function(attempts300, y_fit_true)
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_valid,
              file_path="../../Figs/curve_fitting/exponential4/single/valid{0}".format(i+1), 
              clt_num=i+1, cost = exp3_cost_valid)
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit_true,
              file_path="../../Figs/curve_fitting/exponential4/single/true{0}".format(i+1), 
              clt_num=i+1, cost = exp3_cost_true)

# 2. Multiple Curves

In [ ]:
for p in range(9): # for 10 pieces
    costs = []
    lengths = np.zeros([1, p+2, 13])
    indice = np.zeros([1, p+2, 13])

    for i in range(3): # for parameters
        print("\n\n - {0} pieces, {1} Parameter".format(p+2, i+2))
        if i == 0: # 2 parameters
            seed = [1, 1]
            leastFunction = cv.exponential_least2
        elif i == 1: # 3 parameters
            seed = [1, 1, 1]
            leastFunction = cv.exponential_least3
        elif i == 2: # 4 parmaeters
            seed = [1, 1, 1, 1]
            leastFunction = cv.exponential_least4

        for j in range(nClt): # for clusters
            print("\t avg{0}, ".format(j+1), end='')
            file_name = "avg{0}".format(j+1)
            cost, min_length, min_indice = cv.multCurve_Fitting(eval(file_name), leastFunction, s=seed, w=30, p=p+2)
            costs.append(cost)
            lengths[:, :, j] = min_length
            indice[:, :, j] = min_indice
            dcp.make_folders('../../data/pickles/Fitting/15Attempts/Exponential/pieces{0}/param{1}/avg{2}/'.format(p+2, i+2, j+1))
            dcp.make_Pickle(cost, '../../data/pickles/Fitting/15Attempts/Exponential/pieces{0}/param{1}/avg{2}/cost'.format(p+2, i+2, j+1))
            dcp.make_Pickle(min_length, '../../data/pickles/Fitting/15Attempts/Exponential/pieces{0}/param{1}/avg{2}/length'.format(p+2, i+2, j+1))
            dcp.make_Pickle(min_indice, '../../data/pickles/Fitting/15Attempts/Exponential//pieces{0}/param{1}/avg{2}/index'.format(p+2, i+2, j+1))



 - 2 pieces, 2 Parameter
	 avg1, 	 avg2, 	 avg3, 	 avg4, 	 avg5, 	 avg6, 	 avg7, 	 avg8, 	 avg9, 	 avg10, 	 avg11, 	 avg12, 

 - 2 pieces, 3 Parameter
	 avg1, 	 avg2, 	 avg3, 	 avg4, 	 avg5, 	 avg6, 	 avg7, 	 avg8, 	 avg9, 	 avg10, 	 avg11, 	 avg12, 

 - 2 pieces, 4 Parameter
	 avg1, 	 avg2, 	 avg3, 	 avg4, 	 avg5, 	 avg6, 	 avg7, 	 avg8, 	 avg9, 	 avg10, 	 avg11, 	 avg12, 

 - 3 pieces, 2 Parameter
	 avg1, 	 avg2, 	 avg3, 	 avg4, 	 avg5, 

#### 2) three curves

## 2-2. Three parameters
#### 1) two curves

#### 2) three curves

## 2-2. Four parameters
#### 1) two curves

#### 2) three curves